In [64]:
%config IPCompleter.greedy=True

#Welcome to the big QQQ notebook

In [65]:
#imports
import pandas as pd
import time

In [66]:
#import data
qqq_day = pd.read_csv('latest_data/QQQ_1D_AUG2820.csv')
qqq_5m = pd.read_csv('latest_data/QQQ_5M_AUG2820.csv')
qqq_30m = pd.read_csv('latest_data/QQQ_30M_AUG2820.csv')

#constants
PROFIT_THRESHOLD = 0.002
BIG_PROFIT_THRESHOLD = 0.015
DAYS_OF_THE_WEEK = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

#utilities
def convert_epoch_time_to_day_of_the_week(epoch_milliseconds):
    epoch = epoch_milliseconds / 1000
    return time.strftime("%A", time.gmtime(epoch_milliseconds))

In [67]:
#PROCESS DATA
def ProcessData(dat):
    for i, row in dat.iterrows():
        if i == 0:
            continue
        #add day
        dat.at[i,'dayofweek'] = convert_epoch_time_to_day_of_the_week(dat.at[i, 'time'])
        #add profitability
        if row['close'] / dat.at[i-1, 'close'] - 1 > PROFIT_THRESHOLD:
            dat.at[i,'profit'] = 1
        else:
            dat.at[i,'profit'] = 0
        if row['close'] / dat.at[i-1, 'close'] - 1 > BIG_PROFIT_THRESHOLD:
            dat.at[i,'bigprofit'] = 1
        else:
            dat.at[i,'bigprofit'] = 0
        #rsi over/undersold
        #bb over/undersold
        #market bottom
    return dat

qqq_day = ProcessData(qqq_day)
qqq_30m = ProcessData(qqq_30m)
qqq_5m = ProcessData(qqq_5m)

In [68]:
#DATASET SUMMARY
#columns
qqq_day.columns

Index(['time', 'open', 'high', 'low', 'close', 'Basis', 'Upper', 'Lower',
       'Volume', 'Volume MA', 'Plot', 'Plot.1', 'Range High Percentile',
       'Range High Percentile.1', 'Williams Vix Fix', 'Upper Band', 'RSI',
       'Aroon Up', 'Aroon Down', 'dayofweek', 'profit', 'bigprofit'],
      dtype='object')

In [95]:
#PROFITABLE INTRADAY MOVEMENT
#parameters
PB_DAY_WINDOW = 21

def GetProfitableDayCount(dat, day, window):
    daycnt = 0
    pcnt = 0
    dat_window = dat.tail(window)
    for i, row in dat_window.iterrows():
        if dat.at[i, 'dayofweek'] == day:
            daycnt += 1
            if dat.at[i, 'profit'] == 1:
                pcnt += 1
    return daycnt, pcnt, pcnt/daycnt

result_profit_probability_per_day = []
for day in DAYS_OF_THE_WEEK:
    result_profit_probability_per_day.append(GetProfitableDayCount(qqq_day, day, PB_DAY_WINDOW))

#print summary
i = 0
for day in DAYS_OF_THE_WEEK:
    print (day + " probability of profitable movement (" + str(PB_DAY_WINDOW) + " day window): " + str(result_profit_probability_per_day[i][2]))
    i += 1

Monday probability of profitable movement (21 day window): 0.75
Tuesday probability of profitable movement (21 day window): 0.75
Wednesday probability of profitable movement (21 day window): 0.75
Thursday probability of profitable movement (21 day window): 0.75
Friday probability of profitable movement (21 day window): 0.6


In [86]:
#DAILY SWINGS
EVENT_THRESHOLD = 0.004

def GetDayTradeCallOpportunities(dat):
    day = dat.at[0, 'dayofweek']
    days = 1
    events = []
    dcnt = 0
    for i, row in dat.iterrows():
        if day != dat.at[i, 'dayofweek']:
            low = row
            day = dat.at[i, 'dayofweek']  
            days += 1
            dcnt = 0
            continue
            
        dat.at[i, 'callopportunities'] = dcnt
        dat.at[i, 'day'] = days        
        if i == 0:
            low = row
            continue
        
        if dat.at[i, 'close'] < low['close']:
            low = row
            continue
        
        pctchange = ((row['close'] - low['close']) / low['close'])
        if pctchange > EVENT_THRESHOLD:
            events.append((low,row))
            dcnt += 1
            low = row
    return (dat, events)

def GetDayTradePutOpportunities(dat):
    day = dat.at[0, 'dayofweek']
    days = 1
    events = []
    dcnt = 0
    for i, row in dat.iterrows():
        if day != dat.at[i, 'dayofweek']:
            high = row
            day = dat.at[i, 'dayofweek']  
            days += 1
            dcnt = 0
            continue
            
        dat.at[i, 'putopportunities'] = dcnt
        dat.at[i, 'day'] = days        
        if i == 0:
            high = row
            continue
        
        if dat.at[i, 'close'] > high['close']:
            high = row
            continue
        
        pctchange = ((row['close'] - high['close']) / high['close'])
        if abs(pctchange) > EVENT_THRESHOLD:
            events.append((high,row))
            dcnt += 1
            high = row
    return (dat, events)

#print summary
result = GetDayTradeCallOpportunities(qqq_5m)
qqq_5m = result[0]
print (str(EVENT_THRESHOLD * 100) + "% call events per day: " + str(len(result[1]) / qqq_5m['day'].max()))
result = GetDayTradePutOpportunities(qqq_5m)
qqq_5m = result[0]
print (str(EVENT_THRESHOLD * 100) + "% put events per day: " + str(len(result[1]) / qqq_5m['day'].max()))

0.4% call events per day: 3.96
0.4% put events per day: 2.76


In [ ]:
#BIG MOVEMENT EVENTS

In [60]:
#PRECIOUS METALS INDICATORS

In [ ]:
#FAANG (AND OTHER TECH) INDICATORS

In [ ]:
#EXTERNAL INDICATORS